In [1]:
import torch

In [2]:
import numpy as np

In [3]:
import sys
import pathlib

In [4]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [5]:
import matplotlib.pyplot as plt

---

In [6]:
sys.path.insert(0, str(pathlib.Path("../paper02/src")))

In [7]:
# noinspection PyPackageRequirements, PyUnresolvedReferences
from datasets import get_blobs

---

In [8]:
data, target = get_blobs()

# Scikit-Learn Model

In [9]:
model = MLPClassifier(hidden_layer_sizes=(100, ))

In [10]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3)

In [11]:
model.fit(X_train, y_train);

In [12]:
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

0.8796747967479674

# PyTorch Model
[Source](https://medium.com/biaslyai/pytorch-introduction-to-neural-network-feedforward-neural-network-model-e7231cff47cb)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3)

X_train = torch.FloatTensor(X_train.values)
X_test = torch.FloatTensor(X_test.values)
y_train = torch.FloatTensor(y_train.values > 0)
y_test = torch.FloatTensor(y_test.values > 0)

In [14]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

(torch.Size([1433, 2]),
 torch.Size([615, 2]),
 torch.Size([1433]),
 torch.Size([615]))

In [15]:
class FeedForward(torch.nn.Module):
    def __init__(self, input_size, hidden_size):
        super(FeedForward, self).__init__()

        self.input_size = input_size
        self.hidden_size  = hidden_size
        self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)
        self.activation = torch.nn.modules.activation.Tanh()
        self.fc2 = torch.nn.Linear(self.hidden_size, 1)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        hidden = self.fc1(x)
        activation = self.activation(hidden)
        output = self.fc2(activation)
        output = self.sigmoid(output)
        return output

In [18]:
model = FeedForward(input_size=X_train.shape[1], hidden_size=100)
list(model.parameters())

[Parameter containing:
 tensor([[-0.3442,  0.1052],
         [-0.1196,  0.3383],
         [ 0.5783,  0.6527],
         [-0.1986, -0.4107],
         [ 0.6552, -0.7043],
         [-0.1917, -0.7037],
         [ 0.1366,  0.0353],
         [-0.0945,  0.2322],
         [-0.4663, -0.1278],
         [-0.5419, -0.4661],
         [ 0.3972, -0.2507],
         [ 0.0249, -0.1457],
         [-0.5649, -0.4915],
         [ 0.4690,  0.6167],
         [-0.6890, -0.1799],
         [-0.3887, -0.7037],
         [ 0.6419, -0.2280],
         [ 0.2610, -0.0375],
         [-0.5586, -0.0326],
         [ 0.4489,  0.2585],
         [ 0.3796, -0.6680],
         [-0.3190,  0.5998],
         [-0.5103,  0.1720],
         [-0.3876, -0.3718],
         [ 0.5493,  0.3481],
         [-0.2065, -0.5005],
         [ 0.2198,  0.0280],
         [-0.2571, -0.3262],
         [-0.3754,  0.4973],
         [-0.3492, -0.3015],
         [-0.0764, -0.6834],
         [-0.4980, -0.1697],
         [-0.3329, -0.5753],
         [ 0.3919,  

In [ ]:
model = FeedForward(input_size=X_train.shape[1], hidden_size=100)
criterion = torch.nn.modules.loss.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
model.train()
epoch = 200

for epoch in range(epoch):
    optimizer.zero_grad()
    y_pred = model(X_train)
    loss = criterion(y_pred.squeeze(), y_train)
    loss.backward()
    optimizer.step()

In [ ]:
model.eval()
y_pred = model(X_test)
after_train = criterion(y_pred.squeeze(), y_test)

print('Test loss after Training' , after_train.item())

In [ ]:
plt.hist(y_pred.detach().numpy(), alpha=.5)
plt.hist(y_test.detach().numpy(), alpha=.5)
plt.show()
plt.close()

In [ ]:
accuracy_score(y_pred.detach().numpy() > 0.5, y_test.detach().numpy() > 0.5)

---